## 3D Implied Volatility Surface Download market data directly via `yfinance` and visualize the implied volatility surface.

In [62]:
import numpy as np import pandas as pd import plotly.graph_objects as go import datetime as dt import yfinance as yf  pd.options.display.float_format = lambda x: f'{x:.4f}' 

In [63]:
TICKER = 'SPY' MAX_EXPIRATIONS = None  # set to an int to limit number of monthly expirations  def fetch_spot(ticker: yf.Ticker) -> float:     history = ticker.history(period='1d')     if history.empty:         raise RuntimeError('Unable to retrieve spot price.')     return float(history['Close'].iloc[-1])  def select_monthly_expirations(expirations):     today = dt.datetime.utcnow().date()     limit_date = today + dt.timedelta(days=365)     monthly = {}     for exp in expirations:         exp_date = dt.datetime.strptime(exp, '%Y-%m-%d').date()         if not (today < exp_date <= limit_date):             continue         key = (exp_date.year, exp_date.month)         if key not in monthly or exp_date < monthly[key][0]:             monthly[key] = (exp_date, exp)     return sorted([(dt.datetime.combine(date, dt.time()), label) for date, label in monthly.values()])  ticker = yf.Ticker(TICKER) spot = fetch_spot(ticker) expirations = ticker.options if not expirations:     raise RuntimeError(f'No option expirations found for {TICKER}') selected = select_monthly_expirations(expirations) if MAX_EXPIRATIONS is not None:     selected = selected[:MAX_EXPIRATIONS] if not selected:     raise RuntimeError('No expirations found within the next year.') rows = [] now = dt.datetime.utcnow() for expiry_dt, expiry_str in selected:     T = max((expiry_dt - now).total_seconds() / (365.0 * 24 * 3600), 0.0)     chain = ticker.option_chain(expiry_str).calls     for _, row in chain.iterrows():         rows.append({             'S0': spot,             'K': float(row['strike']),             'T': T,             'C_mkt': float(row['lastPrice']),             'iv': float(row['impliedVolatility']),         }) df_raw = pd.DataFrame(rows) print(f'Fetched {len(df_raw)} rows for {TICKER}') 

/tmp/ipykernel_17684/684805819.py:11: DeprecationWarning:  datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).  /tmp/ipykernel_17684/684805819.py:34: DeprecationWarning:  datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).  

Fetched 895 rows for SPY 

In [64]:
required_cols = {'S0', 'K', 'T', 'C_mkt', 'iv'} missing = required_cols - set(df_raw.columns) if missing:     raise ValueError(f'Input data is missing columns: {missing}')  spot = float(df_raw['S0'].median()) lower_bound = np.ceil((spot - 100.0) / 10.0) * 10.0 upper_bound = np.ceil((spot + 100.0) / 10.0) * 10.0 mask = (df_raw['K'] >= lower_bound) & (df_raw['K'] <= upper_bound) df = df_raw.loc[mask].sort_values(['T', 'K']).reset_index(drop=True) if df.empty:     raise ValueError('No strikes within the specified window.')  print(f'Spot ~ {spot:.2f}. Keeping strikes in [{lower_bound:.2f}, {upper_bound:.2f}] => {len(df)} rows.') 

Spot ~ 671.93. Keeping strikes in [580.00, 780.00] => 489 rows. 

In [65]:
k_values = np.sort(df['K'].unique()) t_values = np.sort(df['T'].unique()) surface = df.pivot_table(index='T', columns='K', values='iv', aggfunc='mean') surface = surface.reindex(index=t_values, columns=k_values) surface = surface.interpolate(axis=1, limit_direction='both').interpolate(axis=1, limit_direction='both') surface = surface.loc[surface.index >= 0.1] IV_surface = surface.to_numpy(dtype=float) if np.isnan(IV_surface).any():     IV_surface = np.where(np.isnan(IV_surface), np.nanmean(IV_surface), IV_surface) k_values = surface.columns.to_numpy(dtype=float) t_values = surface.index.to_numpy(dtype=float) KK, TT = np.meshgrid(k_values, t_values) fig = go.Figure(     data=[         go.Surface(             x=KK,             y=TT,             z=IV_surface,             colorscale='Viridis',             colorbar=dict(title='IV'),             showscale=True,         )     ] ) fig.update_layout(     title='Interpolated Implied Volatility Surface (S0 ± 100 strikes)',     scene=dict(         xaxis=dict(title=dict(text='Strike K — <span style="color:#ff0000">S0</span>')),         yaxis=dict(title='Time to Maturity T (years)'),         zaxis=dict(title='Implied Volatility'),     ),     width=900,     height=600, ) fig.show() 